In [2]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import torch

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from train import setup_dataloaders, setup_experiment, build_env, do_train

In [4]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [12]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model:
RotoTransNetMLP(
  (roto_extractor): Sequential(
    (0): View()
    (1): MLPResNetBlock(
      (model): Sequential(
        (0): Linear(in_features=68, out_features=16, bias=False)
        (1): LeakyReLU(negative_slope=0.01)
        (2): ResNetBlock(
          (layer1): Linear(in_features=16, out_features=16, bias=False)
          (layer2): Linear(in_features=16, out_features=16, bias=False)
          (activation): LeakyReLU(negative_slope=0.01)
        )
        (3): Linear(in_features=16, out_features=6, bias=False)
      )
    )
  )
  (trans_extractor): Sequential(
    (0): View()
    (1): MLPResNetBlock(
      (model): Sequential(
        (0): Linear(in_features=68, out_features=16, bias=False)
        (1): LeakyReLU(negative_slope=0.01)
        (2): ResNetBlock(
          (layer1): Linear(in_features=16, out_features=16, bias=False)
          (layer2): Linear(in_features=16, out_features=16, bias=False)
          (activation): LeakyReLU(negative_slope=0.01)
       

In [ ]:
from torchviz import make_dot